<a href="https://colab.research.google.com/github/Pablobarchiesi94/PI_1_MLOps/blob/main/API/Pruebas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
!pip install fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [4]:
from fastapi import FastAPI
from typing import Optional

app = FastAPI()

In [5]:
# ABRO DATAFRAME FINAL LIMPIO
ruta_archivo = '/content/drive/MyDrive/df_final_limpio.csv'
# LeO el archivo CSV y creO un DataFrame final limpio
df_final = pd.read_csv(ruta_archivo)

In [6]:
@app.get('/cantidad_filmaciones_mes')
def cantidad_filmaciones_mes(Mes: str):
    mes_lower = Mes.lower()
    # Cuento la cantidad de películas estrenadas en el mes
    cantidad = df_final[df_final['Mes'].str.lower() == mes_lower].shape[0]
    return f"{cantidad} cantidad de películas fueron estrenadas en el mes de {Mes}"

In [7]:
# Pruebo de la función
resultado = cantidad_filmaciones_mes('enero')
print(resultado)


5226 cantidad de películas fueron estrenadas en el mes de enero


In [8]:
# Pruebo de la función
resultado = cantidad_filmaciones_mes('Agosto')
print(resultado)

3260 cantidad de películas fueron estrenadas en el mes de Agosto


In [9]:
@app.get('/cantidad_filmaciones_dia')
def cantidad_filmaciones_dia(Dia: str):
    dia_lower = Dia.lower()
    #Cuento la cantidad de películas estrenadas en el día
    cantidad = df_final[df_final['DiaSemana'].str.lower() == dia_lower].shape[0]
    return f"{cantidad} cantidad de películas fueron estrenadas en los días {Dia}"


In [10]:
# Pruebo de la función
resultado = cantidad_filmaciones_dia('JUEVES')
print(resultado)

7119 cantidad de películas fueron estrenadas en los días JUEVES


In [11]:
@app.get('/score_titulo')
def score_titulo(titulo_de_la_filmación: str):
    titulo_lower = titulo_de_la_filmación.lower()
    # Obtengo el título, año de estreno y score
    row = df_final[df_final['title'].str.lower() == titulo_lower].iloc[0]
    titulo = row['title']
    año = row['release_year']
    score = row['popularity']
    return f"La película {titulo} fue estrenada en el año {año} con un score/popularidad de {score}"


In [12]:
# Pruebo de la función
resultado = score_titulo('madagascar')
print(resultado)

La película Madagascar fue estrenada en el año 2005 con un score/popularidad de 16.136217


In [13]:
@app.get('/votos_titulo')
def votos_titulo(titulo_de_la_filmación: str):
    titulo_lower = titulo_de_la_filmación.lower()
    # Obtengo el título, cantidad de votos y valor promedio
    row = df_final[df_final['title'].str.lower() == titulo_lower].iloc[0]
    titulo = row['title']
    año = row['release_year']
    cantidad_votos = row['vote_count']
    promedio_votos = row['vote_average']

    if cantidad_votos < 2000:
        return "La película no cumple la condición de tener al menos 2000 valoraciones."
    else:
        return f"La película {titulo} fue estrenada en el año {año}. La misma cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {promedio_votos}"



In [14]:
# Pruebo de la función
resultado = votos_titulo('Grumpier Old Men')
print(resultado)

La película no cumple la condición de tener al menos 2000 valoraciones.


In [15]:
# Pruebo de la función
resultado = votos_titulo('Toy Story')
print(resultado)

La película Toy Story fue estrenada en el año 1995. La misma cuenta con un total de 5415.0 valoraciones, con un promedio de 7.7


In [16]:
@app.get('/get_actor')
def get_actor(nombre_actor: str):
    nombre_actor_lower = nombre_actor.lower()
    # Relleno los valores NaN en las columnas relevantes con valores adecuados para que no largue error
    df_final['cast'] = df_final['cast'].fillna('')
    df_final['return'] = df_final['return'].fillna(0)
    # Obtengo el éxito del actor, cantidad de filmaciones y promedio de retorno
    actor_data = df_final[df_final['cast'].str.lower().str.contains(nombre_actor_lower, case=False)]
    cantidad_filmaciones = actor_data.shape[0]
    éxito = actor_data['return'].sum()
    promedio_retorno = éxito / cantidad_filmaciones if cantidad_filmaciones > 0 else 0

    return f"El actor {nombre_actor} ha participado de {cantidad_filmaciones} filmaciones. El mismo ha conseguido un retorno de {éxito} con un promedio de {promedio_retorno} por filmación"


In [17]:
# Pruebo de la función
resultado = get_actor('george clooney')
print(resultado)


El actor george clooney ha participado de 49 filmaciones. El mismo ha conseguido un retorno de 106.41996987784233 con un promedio de 2.171836119955966 por filmación


In [18]:
@app.get('/get_director')
def get_director(nombre_director: str):
    nombre_director = nombre_director.lower()
    # Relleno los valores NaN en las columnas relevantes con valores adecuados para que no largue error
    df_final['crew'] = df_final['crew'].fillna('')
    df_final['return'] = df_final['return'].fillna(0)
    # Obtengo el éxito del director, detalles de las películas, costo y ganancia
    director_data = df_final[df_final['crew'].str.contains(nombre_director, case=False)]
    éxito = director_data['return'].sum()
    peliculas = []

    for _, row in director_data.iterrows():
        titulo = row['title']
        fecha_lanzamiento = row['release_date']
        retorno_individual = row['return']
        costo = row['budget']
        ganancia = row['revenue']
        peliculas.append({
            'Título': titulo,
            'Fecha de lanzamiento': fecha_lanzamiento,
            'Retorno individual': retorno_individual,
            'Costo': costo,
            'Ganancia': ganancia
        })

    return {
        'Éxito': éxito,
        'Películas': peliculas
    }

In [19]:
# Pruebo de la función
resultado = get_director('Peter Jackson')
print(resultado)

{'Éxito': 60.39347663449418, 'Películas': [{'Título': 'Heavenly Creatures', 'Fecha de lanzamiento': '1994-09-02', 'Retorno individual': 0.609827, 'Costo': 5000000.0, 'Ganancia': 3049135.0}, {'Título': 'The Frighteners', 'Fecha de lanzamiento': '1996-07-18', 'Retorno individual': 0.9786405333333332, 'Costo': 30000000.0, 'Ganancia': 29359216.0}, {'Título': 'Braindead', 'Fecha de lanzamiento': '1992-08-13', 'Retorno individual': 0.0808743333333333, 'Costo': 3000000.0, 'Ganancia': 242623.0}, {'Título': 'Bad Taste', 'Fecha de lanzamiento': '1987-12-01', 'Retorno individual': 0.0, 'Costo': 25710.0, 'Ganancia': 0.0}, {'Título': 'Forgotten Silver', 'Fecha de lanzamiento': '1995-10-28', 'Retorno individual': 0.0, 'Costo': 650000.0, 'Ganancia': 0.0}, {'Título': 'Meet the Feebles', 'Fecha de lanzamiento': '1989-08-12', 'Retorno individual': 0.0, 'Costo': 750000.0, 'Ganancia': 0.0}, {'Título': 'The Lord of the Rings: The Fellowship of the Ring', 'Fecha de lanzamiento': '2001-12-18', 'Retorno indiv